### 바이트 코드 확인하기

In [1]:
import dis

In [2]:
dis.dis('a = 3')

  1           0 LOAD_CONST               0 (3)
              2 STORE_NAME               0 (a)
              4 LOAD_CONST               1 (None)
              6 RETURN_VALUE


### 순환을 실행할 경우 실행 시간 확인하기

In [3]:
N = 10000000

#### 주피터노트북 명령어 사용하기 

In [4]:
%%time
def w():
    i = 0
    result = []
    while i < N:
        result.append(i)
        i += 1
        
w()

Wall time: 2.44 s


#### 함수 내부 로직을 바이트 코드로 보기

In [5]:
dis.dis(w)

  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               0 (i)

  3           4 BUILD_LIST               0
              6 STORE_FAST               1 (result)

  4           8 SETUP_LOOP              30 (to 40)
        >>   10 LOAD_FAST                0 (i)
             12 LOAD_GLOBAL              0 (N)
             14 COMPARE_OP               0 (<)
             16 POP_JUMP_IF_FALSE       38

  5          18 LOAD_FAST                1 (result)
             20 LOAD_ATTR                1 (append)
             22 LOAD_FAST                0 (i)
             24 CALL_FUNCTION            1
             26 POP_TOP

  6          28 LOAD_FAST                0 (i)
             30 LOAD_CONST               2 (1)
             32 INPLACE_ADD
             34 STORE_FAST               0 (i)
             36 JUMP_ABSOLUTE           10
        >>   38 POP_BLOCK
        >>   40 LOAD_CONST               0 (None)
             42 RETURN_VALUE


In [6]:
%%time
result = []
def a():
    for i in range(N):
        result.append(i)
        
a()

Wall time: 1.76 s


In [7]:
dis.dis(a)

  3           0 SETUP_LOOP              26 (to 28)
              2 LOAD_GLOBAL              0 (range)
              4 LOAD_GLOBAL              1 (N)
              6 CALL_FUNCTION            1
              8 GET_ITER
        >>   10 FOR_ITER                14 (to 26)
             12 STORE_FAST               0 (i)

  4          14 LOAD_GLOBAL              2 (result)
             16 LOAD_ATTR                3 (append)
             18 LOAD_FAST                0 (i)
             20 CALL_FUNCTION            1
             22 POP_TOP
             24 JUMP_ABSOLUTE           10
        >>   26 POP_BLOCK
        >>   28 LOAD_CONST               0 (None)
             30 RETURN_VALUE


In [8]:
%%time
def b():
    result = [ x for x in range(N)]
    
b()

Wall time: 1.1 s


In [9]:
dis.dis(b)

  2           0 LOAD_CONST               1 (<code object <listcomp> at 0x0000000006547AE0, file "<timed exec>", line 2>)
              2 LOAD_CONST               2 ('b.<locals>.<listcomp>')
              4 MAKE_FUNCTION            0
              6 LOAD_GLOBAL              0 (range)
              8 LOAD_GLOBAL              1 (N)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 STORE_FAST               0 (result)
             18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


#### 컴프리헨션과 언패키 처리 알아보기

In [10]:
range(N)

range(0, 10000000)

In [11]:
[range(N)]

[range(0, 10000000)]

In [12]:
abc_com = [x for x in range(N)]

In [13]:
abc = [*range(N)]

In [14]:
len(abc)

10000000

In [15]:
%%time
def c():
    result = [*range(N)]
    
c()

Wall time: 9.14 s


In [16]:
dis.dis(c)

  2           0 LOAD_GLOBAL              0 (range)
              2 LOAD_GLOBAL              1 (N)
              4 CALL_FUNCTION            1
              6 BUILD_LIST_UNPACK        1
              8 STORE_FAST               0 (result)
             10 LOAD_CONST               0 (None)
             12 RETURN_VALUE


### map filter reduce 처리

In [17]:
from inspect import getsource

In [18]:
def map_(func, iterable):
    for el in iterable:
        yield func(el)

In [19]:
map_(lambda x : x+2, range(10))

<generator object map_ at 0x000000000654B4C0>

In [20]:
[ *map_(lambda x : x+2, range(10))]

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [21]:
[ *map(lambda x : x+2, range(10))]

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [22]:
def filter_(func, iterable):
    for el in iterable:
        if func(el):
            yield el

In [23]:
[ *filter_(lambda x : x > 3, [1, 2, 3, 4, 5]) ]

[4, 5]

In [24]:
[ *filter(lambda x : x > 3, [1, 2, 3, 4, 5]) ]

[4, 5]

In [25]:
def reduce_(func, iterable):
    if len(iterable) == 1:
        return iterable[0]
    
    result = func(iterable[0], iterable[1])
    
    return reduce_(func, [result] + iterable[2:])

In [26]:
reduce_(lambda x, y : x / y, [1, 2, 3, 4, 5])

0.008333333333333333

In [27]:
import functools as ft

In [28]:
ft.reduce(lambda x, y : x / y, [1, 2, 3, 4, 5])

0.008333333333333333

## "28+32+++32++39"

# 1. 명령형
# 2. 함수형 -> map, reduce, filter

In [29]:
strings = "28+32+++32++39"

In [30]:
strings.split('+')

['28', '32', '', '', '32', '', '39']

In [31]:
# 명령형
result =  0
for string in strings.split("+"):
    if string != "": 
        result += int(string)

print(result)
#strings.split("+")

131


In [32]:
bool(0), bool(''), bool([])

(False, False, False)

In [33]:
# False [], {}, set(), '', 0

[ *map(int, filter(bool, strings.split("+"))) ]

[28, 32, 32, 39]

In [34]:
from functools import reduce

In [35]:
# map ,filter 1개씩
# reduce 2개씩
reduce(lambda x, y: x+y, [1,2,3,4,5]) # 

15

In [36]:
[ *filter(lambda x : x % 2 ==0 , [1,2,3,4,5]) ] 

[2, 4]

In [37]:

from operator import add

reduce(add, map(int, filter(bool, strings.split("+"))))  # 함수의 함성

131